# Instruction

**Run each cell one by one**

# Load data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install mne



import numpy as np
import scipy.io as sio
import mne
def indexget(total,select):
  return [i for i, e in enumerate(total) if e in select]



def inputmat(fp1,fp2,fp3):
    """load .mat file and return m as a dict"""
    mat = sio.loadmat(fp1, squeeze_me=True)
    label_mat=sio.loadmat(fp2, squeeze_me=True)
    m = {}  # create a dict

    # Numpy array of size channel_num * points.
    select_channel=fp3
    channelindex= indexget(mat['nfo']['clab'][True][0],select_channel)
    m['ch_names'] = mat['nfo']['clab'][True][0][channelindex]
    


    m['data'] = mat['cnt'].T[channelindex]
    m['freq'] = mat['nfo']['fs'][True][0]  # Sampling frequency

    # channel names are necessary information for creating a rawArray.


    # Position of channels
    m['electrode_x'] = mat['nfo']['xpos'][True][0]
    m['electrode_y'] = mat['nfo']['ypos'][True][0]

    # find trials and its data
    m['cue'] = mat['mrk']['pos'][True][0]  # time of cue
    m['labels'] = label_mat['true_y']  
    # m['labels'] = np.nan_to_num(mat['mrk']['y'][True][0]).astype(int)  # convert NaN to 0
    m['test_idx'] = len(label_mat['test_idx'])
    m['n_trials'] = len(m['labels'])  # Number of the total useful trials
    return m


def creatEventsArray(fp1,fp2,fp3):
    """Create events array. The second column default to zero."""
    m = inputmat(fp1,fp2,fp3)
    events = np.zeros((m['n_trials'], 3), int)
    events[:, 0] = m['cue'][:m['n_trials']]  # The first column is the sample number of the event.
    events[:, 2] = m['labels'][:m['n_trials']]  # The third column is the new event value.
    return events, m['labels']


def creatRawArray(fp1,fp2,fp3):
    """Create a mne.io.RawArray object, data: array, shape (n_channels, n_times)"""
    m = inputmat(fp1,fp2,fp3)
    ch_names = m['ch_names'].tolist()
    info = mne.create_info(ch_names, m['freq'], 'eeg')  # Create info for raw
    raw = mne.io.RawArray(m['data'], info, first_samp=0, copy='auto', verbose=None)
    return raw

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.5 MB 4.4 MB/s 


# Load Package

In [2]:
import numpy as np
import scipy.io as sio
from tensorflow.keras.utils import to_categorical
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd.function import Function
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader,TensorDataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plot
from sklearn.decomposition import PCA
import torch.utils.data as Data
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from torchvision import models
from torchsummary import summary
import torch.optim.lr_scheduler as lr_scheduler


In [3]:
print(__doc__)

fp = {
    'aa': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/data_set_IVa_aa.mat',
    'al': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/data_set_IVa_al.mat',
    'av': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/data_set_IVa_av.mat',
    'aw': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/data_set_IVa_aw.mat',
    'ay': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/data_set_IVa_ay.mat',
}


fplabel={
    'aa': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/true_labels_aa.mat',
    'al': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/true_labels_al.mat',
    'av': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/true_labels_av.mat',
    'aw': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/true_labels_aw.mat',
    'ay': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/true_labels_ay.mat',

}

channel_set=['C1',  'C3',  'Cz',  'C2',  'C4',  'CFC3',  'CFC4','CFC5',  'CFC6',  'CCP3', 
             'CCP4',  'CCP5',  'CCP6',  'T7',  'T8',  'P3',  'Pz',  'P4']
pick_chan = {
    'aa':  channel_set,
    'al':  channel_set,
    'av':  channel_set,
    'aw':  channel_set,
    'ay':  channel_set,

}

low_freq, high_freq = 7., 30.
tmin, tmax = 0., 3.5
# event_id
event_id = {'right': 1, 'foot': 2}

Automatically created module for IPython interactive environment


# Processing the data

In [4]:
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline

from mne.decoding import CSP
from mne.channels import read_layout



def cal_acc(pred,real):
  return [1 if pred[j]==real[j] else 0 for j in range(len(real))]

def getdata_label(a,b,c):
  mat_dic=inputmat(a,b,c)
  raw = creatRawArray(a,b,c)
  events, labels = creatEventsArray(a,b,c)


    # Apply band-pass filter
  raw.filter(low_freq, high_freq, fir_design='firwin', skip_by_annotation='edge')

    # event_train = eventsTrain(fp[f])
  epochs = mne.Epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True,
                        verbose=False)

  epochs_train = epochs.copy().crop(tmin=0.5, tmax=2.5)
  labels = epochs.events[:, -1]

    # # Define a monte-carlo cross-validation generator (reduce variance):
  epochs_data= epochs_train.get_data()
  return epochs_data,labels,mat_dic['test_idx']

intial_dic={}

total_length=280
train_num=10
test_num=10

for f,fla,cs in zip(fp,fplabel,pick_chan):
    x,y,testix=getdata_label(fp[f],fplabel[fla],pick_chan[cs])

    intial_dic[f]=[x,y]
    # # Assemble a classifier
    # 
    # csp = CSP(n_components=len(epochs.ch_names), reg=None, log=True, norm_trace=False)





Creating RawArray with float64 data, n_channels=18, n_times=298458
    Range : 0 ... 298457 =      0.000 ...  2984.570 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 165 samples (1.650 sec)

Creating RawArray with float64 data, n_channels=18, n_times=283574
    Range : 0 ... 283573 =      0.000 ...  2835.730 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal 

In [5]:
dic=[]
str_list=['aa','al','av','aw','ay']


for i in str_list:
  temp=[]
  x,y=intial_dic[i]
  x=x.reshape(x.shape[0],1,x.shape[1],-1)

  y=y-1
  temp.append(x)
  temp.append(y)

  dic.append(temp)

# Data Load

In [6]:
def train(Subject_number):

#Prepare the raw feature data for training and testing#
  #You can change the "Subject_number" (0-8 corresponding to B01-B09, respectively) to try training and testing between different subjects
  Subject_number=Subject_number
  BATCH_SIZE = 64
  total_x,total_y=dic[Subject_number][0],dic[Subject_number][1]
  hyperparameter=10
  # total_y_onehot=np.vstack((y_train_onehot,y_test_onehot))
  index1=np.where(total_y == 0)[0]
  index2=np.where(total_y == 1)[0]
  choice1=np.random.choice(index1,hyperparameter,replace=False)
  choice2=np.random.choice(index2,hyperparameter,replace=False)
  choice_total=np.hstack((choice1,choice2))
  xxxtest= np.delete(total_x, choice_total,0)
  yyytest=np.delete(total_y, choice_total)


  target_x=total_x[choice_total]
  target_y=total_y[choice_total]



  total_x_train=[]
  total_y_train=[]
  full_subjectnum=[0,1,2,3,4]
  full_subjectnum.remove(Subject_number)
  for train_subject in full_subjectnum:
    x_train,y_train = dic[train_subject][0],dic[train_subject][1]

    if len(total_x_train)==0:
      total_x_train=x_train
      total_y_train=y_train
    else:
      total_x_train=np.vstack((total_x_train,x_train))
      total_y_train=np.hstack((total_y_train,y_train))

  x_train=total_x_train
  y_train=total_y_train


  x_test=xxxtest
  y_test=yyytest




  target_x=torch.from_numpy(target_x)
  target_y=torch.from_numpy(target_y)
  target_x=target_x.float()
  target_y=target_y.long()




  x_train=torch.from_numpy(x_train)
  y_train=torch.from_numpy(y_train)
  x_test=torch.from_numpy(x_test)
  y_test=torch.from_numpy(y_test)
  x_train=x_train.float()
  x_test=x_test.float()
  y_train=y_train.long()
  y_test=y_test.long()
  source_dataset=Data.TensorDataset(x_train,y_train)
  target_dataset=Data.TensorDataset(target_x,target_y)
  val_dataset=Data.TensorDataset(x_test,y_test)


    
  source_loader= Data.DataLoader(source_dataset, batch_size=128, shuffle=True, num_workers=2)
  target_loader = Data.DataLoader(target_dataset, batch_size=2, shuffle=True, num_workers=2)
  test_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)


  return source_loader,target_loader,test_loader

# Network

In [7]:
from torch.autograd import Function


class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None


class Constrainedlinear(nn.Linear):
    def _max_norm(self,w,max_val=0.25,eps=1e-8):
        norm = w.norm(2, dim=1, keepdim=True)
        desired = torch.clamp(norm, 0, max_val)
        return w * (desired / (eps + norm))
    def forward(self, input):
        return F.linear(input, self._max_norm(self.weight),self.bias)


In [8]:
import torch.nn as nn


class CNNModel(nn.Module):

    def __init__(self,classes_num):
        super(CNNModel, self).__init__()
        self.numC=18

        self.drop_out = 0.2
        
        self.block_1 = nn.Sequential(
            # Pads the input tensor boundaries with zero
            # left, right, up, bottom
            nn.ZeroPad2d((15, 16, 0, 0)),
            nn.Conv2d(
                in_channels=1,          # input shape (1, C, T)
                out_channels=8,         # num_filters
                kernel_size=(1, 32),    # filter size
                bias=False
            ),                          # output shape (8, C, T)
            nn.BatchNorm2d(8)           # output shape (8, C, T)
        )
        
        # block 2 and 3 are implementations of Depthwise Convolution and Separable Convolution
        self.block_2 = nn.Sequential(
            nn.Conv2d(
                in_channels=8,          # input shape (8, C, T)
                out_channels=16,        # num_filters
                kernel_size=(self.numC, 1),    # filter size
                groups=8,
                bias=False
            ),                          # output shape (16, 1, T)
            nn.BatchNorm2d(16),         # output shape (16, 1, T)
            nn.ELU(),
            nn.AvgPool2d((1, 8)),       # output shape (16, 1, T//4)
            nn.Dropout(self.drop_out)   # output shape (16, 1, T//4)
        )
        
        self.block_3 = nn.Sequential(
            nn.ZeroPad2d((7, 8, 0, 0)),
            nn.Conv2d(
               in_channels=16,          # input shape (16, 1, T//4)
               out_channels=16,         # num_filters
               kernel_size=(1, 16),     # filter size
               groups=16,
               bias=False
            ),                          # output shape (16, 1, T//4)
            nn.Conv2d(
                in_channels=16,         # input shape (16, 1, T//4)
                out_channels=16,        # num_filters
                kernel_size=(1, 1),     # filter size
                bias=False
            ),                          # output shape (16, 1, T//4)
            nn.BatchNorm2d(16),         # output shape (16, 1, T//4)
            nn.ELU(),
            nn.AvgPool2d((1, 8)),       # output shape (16, 1, T//32)
            nn.Dropout(self.drop_out)
        )

        self.class_classifier = nn.Sequential()
        self.class_classifier.add_module('c_fc1',  Constrainedlinear(48, classes_num))
        self.class_classifier.add_module('c_softmax', nn.LogSoftmax())




        self.domain_classifier = nn.Sequential()
        self.domain_classifier.add_module('d_fc1', nn.Linear(48, 100))
        self.domain_classifier.add_module('d_bn1', nn.BatchNorm1d(100))
        self.domain_classifier.add_module('d_relu1', nn.ReLU(True))
        self.domain_classifier.add_module('d_fc2', nn.Linear(100, 2))
        self.domain_classifier.add_module('d_softmax', nn.LogSoftmax(dim=1))

    def forward(self, x, alpha):

        x = self.block_1(x)
        x = self.block_2(x)
        x = self.block_3(x)
        
        feature = x.view(x.size(0), -1)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.class_classifier(feature)
        domain_output = self.domain_classifier(reverse_feature)

        return feature, class_output, domain_output

#Center loss

In [9]:
class CenterLoss(nn.Module):
    def __init__(self, num_classes, feat_dim, size_average=True):
        super(CenterLoss, self).__init__()
        self.centers = nn.Parameter(torch.randn(num_classes, feat_dim))
        self.centerlossfunc = CenterlossFunc.apply
        self.feat_dim = feat_dim
        self.size_average = size_average

    def forward(self, label, feat):
        batch_size = feat.size(0)
        feat = feat.view(batch_size, -1)
        # To check the dim of centers and features
        if feat.size(1) != self.feat_dim:
            raise ValueError("Center's dim: {0} should be equal to input feature's \
                            dim: {1}".format(self.feat_dim,feat.size(1)))
        batch_size_tensor = feat.new_empty(1).fill_(batch_size if self.size_average else 1)
        loss = self.centerlossfunc(feat, label, self.centers, batch_size_tensor)
        return loss


class CenterlossFunc(Function):
    @staticmethod
    def forward(ctx, feature, label, centers, batch_size):
        ctx.save_for_backward(feature, label, centers, batch_size)
        centers_batch = centers.index_select(0, label.long())
        return (feature - centers_batch).pow(2).sum() / 2.0 / batch_size

    @staticmethod
    def backward(ctx, grad_output):
        feature, label, centers, batch_size = ctx.saved_tensors
        centers_batch = centers.index_select(0, label.long())
        diff = centers_batch - feature
        # init every iteration
        counts = centers.new_ones(centers.size(0))
        ones = centers.new_ones(label.size(0))
        grad_centers = centers.new_zeros(centers.size())

        counts = counts.scatter_add_(0, label.long(), ones)
        grad_centers.scatter_add_(0, label.unsqueeze(1).expand(feature.size()).long(), diff)
        grad_centers = grad_centers/counts.view(-1, 1)
        return - grad_output * diff / batch_size, None, grad_centers / batch_size, None

#Testing function
def main(test_cuda=False):
    print('-'*80)
    device = torch.device("cuda" if test_cuda else "cpu")
    ct = CenterLoss(10,2,size_average=True).to(device)
    y = torch.Tensor([0,0,2,1]).to(device)
    feat = torch.zeros(4,2).to(device).requires_grad_()
    print (list(ct.parameters()))
    
    print (ct.centers.grad)
    out = ct(y,feat)
    print(out.item())
    out.backward()


In [10]:
import os
import torch.backends.cudnn as cudnn
import torch.utils.data
from torchvision import transforms
from torchvision import datasets


def test(dataset_name,dataloader, epoch):


    cuda = True
    cudnn.benchmark = True
    alpha = 0



    """ training """
    model_root='/'
    my_net = torch.load(os.path.join(
        model_root, 'mnist_mnistm_model_epoch_' + str(epoch) + '.pth'
    ))
    my_net = my_net.eval()

    if cuda:
        my_net = my_net.cuda()

    len_dataloader = len(dataloader)
    data_target_iter = iter(dataloader)

    i = 0
    n_total = 0
    n_correct = 0

    while i < len_dataloader:

        # test model using target data
        data_target = data_target_iter.next()
        t_img, t_label = data_target

        batch_size = len(t_label)

        input_img = torch.FloatTensor(batch_size, 1, 18, 201)
        class_label = torch.LongTensor(batch_size)

        if cuda:
            t_img = t_img.cuda()
            t_label = t_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)
        class_label.resize_as_(t_label).copy_(t_label)

        _,class_output, _ = my_net(x=input_img, alpha=alpha)
        pred = class_output.data.max(1, keepdim=True)[1]
        n_correct += pred.eq(class_label.data.view_as(pred)).cpu().sum()
        n_total += batch_size

        i += 1

    accu = n_correct.data.numpy() * 1.0 / n_total

    print('epoch: %d, accuracy of the %s dataset: %f' % (epoch, dataset_name, accu),n_correct.data.numpy() ,n_total)
    return accu

In [11]:
import os
import torch.backends.cudnn as cudnn
import torch.utils.data
from torchvision import transforms
from torchvision import datasets
import random
import os
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms
import numpy as np

def run(Subject_number):

  source_loader,target_loader,test_loader=train(Subject_number)
 

  cuda = True
  cudnn.benchmark = True
  lr = 1e-3
  n_epoch = 500




  my_net = CNNModel(2)


  centerloss = CenterLoss(2, 48).cuda()
  optimzer4center = optim.Adam(centerloss.parameters(), lr =0.5)
  optimizer = optim.Adam(my_net.parameters(), lr=lr)

  loss_class = torch.nn.NLLLoss()
  loss_domain = torch.nn.NLLLoss()

  if cuda:
    my_net = my_net.cuda()
    loss_class = loss_class.cuda()
    loss_domain = loss_domain.cuda()

  for p in my_net.parameters():
    p.requires_grad = True


  source_loader,target_loader,test_loader=train(Subject_number)
  dataloader_source=source_loader
  dataloader_target=target_loader

  model_root = os.path.join('')
  trainacc=[]
  testacc=[]
  targacc=[]
  for epoch in range(n_epoch):

    len_dataloader = min(len(dataloader_source), len(dataloader_target))
    data_source_iter = iter(dataloader_source)
    data_target_iter = iter(dataloader_target)

    i = 0
    while i < len_dataloader:

        p = float(i + epoch * len_dataloader) / n_epoch / len_dataloader
        alpha = 2. / (1. + np.exp(-10 * p)) - 1

        # training model using source data
        data_source = data_source_iter.next()
        s_img, s_label = data_source




        my_net.zero_grad()
        batch_size = len(s_label)
        input_img = torch.FloatTensor(batch_size, 1, 18, 201)
        


        class_label = torch.LongTensor(batch_size)
        domain_label = torch.zeros(batch_size)
        domain_label = domain_label.long()

        if cuda:
            s_img = s_img.cuda()
            s_label = s_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(s_img).copy_(s_img)
        class_label.resize_as_(s_label).copy_(s_label)


        _,class_output, domain_output = my_net(x=input_img, alpha=alpha)

        err_s_label = loss_class(class_output, class_label)
        err_s_domain = loss_domain(domain_output, domain_label)


        data_target = data_target_iter.next()
        t_img, t_label = data_target

        batch_size = len(t_img)
        class_label = torch.LongTensor(batch_size)


        centerloss.zero_grad()

        input_img = torch.FloatTensor(batch_size, 1, 18, 201)
        domain_label = torch.ones(batch_size)
        domain_label = domain_label.long()

        if cuda:
            t_img = t_img.cuda()
            t_label = t_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)
        class_label.resize_as_(t_label).copy_(t_label)


        fea,class_output, domain_output = my_net(x=input_img, alpha=alpha)

        cens=centerloss(class_label, fea)

        err_t_label = loss_class(class_output, class_label)
        err_t_domain = loss_domain(domain_output, domain_label)
        err = err_t_domain + err_s_domain + 10*err_s_label+err_t_label+0.005*cens

        err.backward()
        optimizer.step()

        # print ('epoch: %d, [iter: %d / all %d], err_s_label: %f, err_s_domain: %f, err_t_domain: %f' \
        #       % (epoch, i, len_dataloader, err_s_label.cpu().data.numpy(),
        #          err_s_domain.cpu().data.numpy(), err_t_domain.cpu().data.numpy()))
    
        i=i+1

    torch.save(my_net, '{0}/mnist_mnistm_model_epoch_{1}.pth'.format(model_root, epoch))
  traac=test('train',dataloader_source, epoch)
  targac=test('target',dataloader_target, epoch)
  tesac=test('test',test_loader, epoch)


In [12]:
Subject_number=2
run(Subject_number)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch: 499, accuracy of the train dataset: 0.800893 897 1120
epoch: 499, accuracy of the target dataset: 0.500000 10 20
epoch: 499, accuracy of the test dataset: 0.526923 137 260
